In [40]:
import sys, os, re; sys.path = (["../src/", "../"] if re.match(r'^(\w\:\\)|(/)', os.getcwd()) else [])+ sys.path 

import ccxt as cx
import ccxt.pro as cxp
from ccxt.base.decimal_to_precision import ROUND_UP
import asyncio
import nest_asyncio
nest_asyncio.apply()

import qubx
%qubxd

%load_ext autoreload
%autoreload 2

from qubx import lookup
from qubx.core.basics import Trade, Quote
from qubx.core.series import TimeSeries
from qubx.impl.ccxt_connector import CCXT_connector

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Common

In [ ]:
lookup.instruments['KRAKEN.F:BTC/USD']

In [34]:
instr = lookup.instruments.find('BINANCE.UM', 'BTC', 'USDT')
instr

Instrument(symbol='BTCUSDT', market_type='CRYPTO', exchange='BINANCE.UM', base='BTC', quote='USDT', margin_symbol='USDT', min_tick=0.1, min_size_step=0.001, min_size=0.001, futures_info=FuturesInfo(contract_type='PERPETUAL', delivery_date=datetime.datetime(2100, 12, 25, 10, 0), onboard_date=datetime.datetime(2019, 9, 25, 11, 0), contract_size=1.0, maint_margin=2.5, required_margin=5.0, liquidation_fee=0.0125), _aux_instrument=None)

In [39]:
instr = lookup.instruments.find('KRAKEN', 'BTC', 'ETH')
instr

Instrument(symbol='ETH/BTC', market_type='spot', exchange='KRAKEN', base='ETH', quote='BTC', margin_symbol=None, min_tick=1e-05, min_size_step=1e-05, min_size=1e-08, futures_info=None, _aux_instrument=None)

In [38]:
lookup.find_aux_instrument_for(instr, 'USD')

Instrument(symbol='BTC/USD', market_type='spot', exchange='KRAKEN', base='BTC', quote='USD', margin_symbol=None, min_tick=0.1, min_size_step=0.1, min_size=1e-08, futures_info=None, _aux_instrument=None)

In [46]:
bu = cxp.binanceusdm()
bs = cxp.binance()
ks = cxp.kraken()
kf = cxp.krakenfutures()

# Get OHLC [TODO]

In [47]:
bs_markets = await bs.load_markets()
bu_markets = await bu.load_markets()
kf_markets = await kf.load_markets()
ks_markets = await ks.load_markets()

In [53]:
bu.parse_timeframe('1m')

60

In [ ]:
interval = bu.parse_timeframe('1m') * 1000
orderbook_max_levels_limit = 100
since = bu.round_timeframe('1m', bu.milliseconds(), ROUND_UP) - orderbook_max_levels_limit * interval
await bu.fetch_ohlcv('ETH/USDT', '1m', since=since, limit=orderbook_max_levels_limit)

In [ ]:
# await bnc1.fetch_ohlcv('ETH/USDT', '1m', since=since, limit=limit)

In [57]:
cxc = CCXT_connector('binance')

# Listen to data

In [50]:
from typing import List
from ccxt.base.exchange import Exchange
from threading import Thread, Event, Lock
from queue import Queue
from multiprocessing import Queue #as Queue

class Channel:
    control: Event
    queue: Queue
    name: str
    lock: Lock

    def __init__(self, name: str):
        self.name = name
        self.control = Event()
        self.queue = Queue()
        self.lock = Lock()

    def stop(self):
        if self.control.is_set():
            self.control.clear()

    def start(self):
        self.control.set()

class RunThread(Thread):
    def __init__(self, channel: Channel):
        self.result = None
        self.ch = channel
        self.loops = []
        super().__init__()

    def add(self, func, *args, **kwargs):
        self.loops.append(func(self.ch, *args, **kwargs))

    async def run_loop(self):
        self.result = await asyncio.gather(*self.loops)

    def run(self):
        self.ch.control.set()
        asyncio.run(self.run_loop())

        # global_loop = asyncio.new_event_loop()
        # # self.result = asyncio.run(self.func(self.ch, *self.args, **self.kwargs))
        # self.result = global_loop.run_until_complete(self.func(self.ch, *self.args, **self.kwargs))

    def stop(self):
        self.ch.control.clear()
        self.ch.queue.put((None, None)) # send sentinel

async def listen_to_trades(channel: Channel, exchange: Exchange, symbols: List[str]):
    method = 'watchTrades'
    if exchange.has[method]:
        lock = channel.lock
        while channel.control.is_set():
            try:
                # c_time = exchange.iso8601(exchange.milliseconds())
                trades = await exchange.watch_trades_for_symbols(symbols)
                # num_trades = len(trades)
                lock.acquire()
                for trade in trades:
                    # trade = trades[-1]
                    t_ns = trade['timestamp'] * 1_000_000 # this is trade time 
                    s = trade['symbol']
                    info = trade['info']
                    price = trade['price']
                    m = info['m']
                    amnt = trade['amount'] # in base currency
                    tas = Trade(t_ns, price, amnt, int(not m), int(trade['id']))
                    channel.queue.put((s, tas))
                lock.release()
                # print(trade, flush=True)
                # print(f"\t[{c_time}] {s} : {str(tas)}", flush=True)
            except Exception as e:
                print(type(e).__name__, str(e), flush=True)
                await exchange.close()
                raise e
    else:
        raise Exception(exchange.id + ' ' + method + ' is not supported or not implemented yet')

async def listen_to_orderbook(channel: Channel, exchange: Exchange, symbols: List[str], orderbook_max_levels_limit=5):
    lock = channel.lock
    while channel.control.is_set():
        try:
            orderbook = await exchange.watch_order_book_for_symbols(symbols, limit=orderbook_max_levels_limit)
            c_time = exchange.iso8601(exchange.milliseconds())
            t_ns = orderbook['timestamp'] * 1_000_000
            s = orderbook['symbol']
            b,a = orderbook['bids'], orderbook['asks']
            q = Quote(t_ns, b[0][0], a[0][0], b[0][1], a[0][1])
            lock.acquire()
            channel.queue.put((s, q))
            lock.release()
            # print(f"[{c_time}] {s} : {str(q)}", flush=True)
        except Exception as e:
            print(type(e).__name__, str(e), flush=True)
            await exchange.close()
            raise e

async def receiver(channel: Channel):
    sers = {}
    print("START LISTENING", flush=True)
    T, Q = 0, 0
    while channel.control.is_set():
        s, data = channel.queue.get()
        if isinstance(data, Quote):
            if not (Q % 100):
                print(s, data, flush=True)
                Q = 0
            Q += 1
        if isinstance(data, Trade):
            if s not in sers:
                sers[s] = TimeSeries(s, '5Min')
            ts = sers[s]
            ts.update(data.time, data.price)

            if not (T % 100):
                print(s, data, flush=True)
                T = 0
            T += 1
    print(f"STOP LISTENING: {str(sers)}", flush=True)
    return sers 

In [51]:
chan = Channel('ticks')

reader = RunThread(chan)
reader.add(receiver)
reader.start()

START LISTENING


In [52]:
bu = cxp.binanceusdm()

In [ ]:
t1 = RunThread(chan)
t1.add(listen_to_trades, bu, ['BTCUSDT', 'ETHUSDT', 'SOLUSDT'])
# t1.add(listen_to_orderbook, bu, ['BTCUSDT', 'ETHUSDT', 'SOLUSDT'])
t1.start()

BTC/USDT:USDT [2024-03-18T16:45:31.820000000]	67708.10000 (0.1) <take> 4758115842
BTC/USDT:USDT [2024-03-18T16:45:32.785000000]	67700.80000 (0.0) <make> 4758115920
BTC/USDT:USDT [2024-03-18T16:45:32.785000000]	67695.20000 (0.3) <make> 4758116020
ETH/USDT:USDT [2024-03-18T16:45:32.821000000]	3542.46000 (0.0) <make> 3770170730
BTC/USDT:USDT [2024-03-18T16:45:33.315000000]	67696.00000 (0.0) <make> 4758116075
ETH/USDT:USDT [2024-03-18T16:45:33.634000000]	3542.22000 (6.8) <make> 3770170802
BTC/USDT:USDT [2024-03-18T16:45:34.026000000]	67690.50000 (0.0) <make> 4758116180
BTC/USDT:USDT [2024-03-18T16:45:34.915000000]	67693.10000 (0.0) <make> 4758116254
BTC/USDT:USDT [2024-03-18T16:45:34.915000000]	67687.20000 (0.0) <make> 4758116319
BTC/USDT:USDT [2024-03-18T16:45:34.915000000]	67679.00000 (0.0) <make> 4758116419
ETH/USDT:USDT [2024-03-18T16:45:35.428000000]	3541.94000 (0.7) <make> 3770170899
ETH/USDT:USDT [2024-03-18T16:45:35.641000000]	3541.41000 (10.0) <make> 3770170999
BTC/USDT:USDT [2024

In [ ]:
t1.stop()

STOP LISTENING


In [55]:
reader.result

[{'BTC/USDT:USDT': 2024-03-18 16:45:00    67681.9
  Name: BTC/USDT:USDT, dtype: float64,
  'ETH/USDT:USDT': 2024-03-18 16:45:00    3541.16
  Name: ETH/USDT:USDT, dtype: float64,
  'SOL/USDT:USDT': 2024-03-18 16:45:00    203.372
  Name: SOL/USDT:USDT, dtype: float64}]

In [48]:
bu.close()

<coroutine object Exchange.close at 0x00000192F75A4DC0>